In [3]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [13]:
GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")

res8: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@73bd795f
